In [67]:
# !pip3 install -r requirements.txt

In [65]:
# !python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --output_dir=model_sep20 --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --eval_on dev

In [19]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from bert import Ner
from run_ner import NerProcessor
import argparse
from model import BertNer
from fastprogress import master_bar, progress_bar
from tokenization import FullTokenizer
from run_ner import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [66]:
# !python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --output_dir=model_sep20 --max_seq_length=128 --num_train_epochs 3 --do_eval --eval_on test

In [69]:
# train_examples = NerProcessor.get_train_examples(args.data_dir)

In [8]:
def split_text_label(filename):
    f = open(filename)
    split_labeled_text = []
    sentence = []
    for line in f:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                split_labeled_text.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        sentence.append([splits[0], splits[-1].rstrip("\n")])
    if len(sentence) > 0:
        split_labeled_text.append(sentence)
        sentence = []
    return split_labeled_text

In [9]:
split_train = split_text_label(os.path.join('data', "train.txt"))
split_valid = split_text_label(os.path.join('data', "valid.txt"))
split_test = split_text_label(os.path.join('data', "test.txt"))

In [10]:
labelSet = set()
wordSet = set()
# words and labels
for data in [split_train, split_valid, split_test]:
    for labeled_text in data:
        for word, label in labeled_text:
            labelSet.add(label)
            wordSet.add(word.lower())

In [11]:
# Sort the set to ensure '0' is assigned to 0
sorted_labels = sorted(list(labelSet), key=len)
# Create mapping for labels
label2Idx = {}
for label in sorted_labels:
    label2Idx[label] = len(label2Idx)
idx2Label = {v: k for k, v in label2Idx.items()}
# Create mapping for words
word2Idx = {}
if len(word2Idx) == 0:
    word2Idx["PADDING_TOKEN"] = len(word2Idx)
    word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
for word in wordSet:
    word2Idx[word] = len(word2Idx)

In [12]:
def createMatrices(data, word2Idx, label2Idx):
    sentences = []
    labels = []
    for split_labeled_text in data:
        wordIndices = []
        labelIndices = []
        for word, label in split_labeled_text:
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]
            else:
                wordIdx = word2Idx['UNKNOWN_TOKEN']
            wordIndices.append(wordIdx)
            labelIndices.append(label2Idx[label])
        sentences.append(wordIndices)
        labels.append(labelIndices)
    return sentences, labels

In [13]:
train_sentences, train_labels = createMatrices(split_train, word2Idx, label2Idx)
valid_sentences, valid_labels = createMatrices(split_valid, word2Idx, label2Idx)
test_sentences, test_labels = createMatrices(split_test, word2Idx, label2Idx)

In [14]:
def padding(sentences, labels, max_len, padding='post'):
    padded_sentences = pad_sequences(sentences, max_len, padding='post')
    padded_labels = pad_sequences(labels, max_len, padding='post')
    return padded_sentences, padded_labels

In [20]:
max_seq_len=16
train_features, train_labels = padding(train_sentences, train_labels, max_seq_len, padding='post')
valid_features, valid_labels = padding(valid_sentences, valid_labels, max_seq_len, padding='post')
test_features, test_labels = padding(test_sentences, test_labels, max_seq_len, padding='post')

In [21]:
# all_input_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.input_ids for f in train_features]))
# all_input_mask = tf.data.Dataset.from_tensor_slices(np.asarray([f.input_mask for f in train_features]))
# all_segment_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.segment_ids for f in train_features]))
# all_valid_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.valid_ids for f in train_features]))
# all_label_mask = tf.data.Dataset.from_tensor_slices(np.asarray([f.label_mask for f in train_features]))
# all_label_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.label_id for f in train_features]))

AttributeError: 'numpy.ndarray' object has no attribute 'input_ids'

In [26]:
# # load tokenizer
# tokenizer = FullTokenizer(os.path.join(args.output_dir, "vocab.txt"), args.do_lower_case)
# # model build hack : fix
# config = json.load(open(os.path.join(args.output_dir, "bert_config.json")))
# ner = BertNer(config, tf.float32, num_labels, 
# args.max_seq_length)
# ids = tf.ones((1,128),dtype=tf.int64)
# _ = ner(ids,ids,ids,ids, training=False)
# ner.load_weights(os.path.join(args.output_dir,"model.h5"))
# # load test or development set based on argsK
# if args.eval_on == "dev":
#     eval_examples = processor.get_dev_examples(args.data_dir)
# elif args.eval_on == "test":
#     eval_examples = processor.get_test_examples(args.data_dir)
#     eval_features = convert_examples_to_features(eval_examples, label_list, args.max_seq_length, tokenizer)
#     all_input_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.input_ids for f in eval_features]))  
#     all_input_mask = tf.data.Dataset.from_tensor_slices(np.asarray([f.input_mask for f in eval_features]))
#     all_segment_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.segment_ids for f in eval_features]))
#     all_valid_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.valid_ids for f in eval_features]))
#     all_label_ids = tf.data.Dataset.from_tensor_slices(np.asarray([f.label_id for f in eval_features]))
#     eval_data = tf.data.Dataset.zip((all_input_ids, all_input_mask, all_segment_ids, all_valid_ids, all_label_ids))
#     batched_eval_data = eval_data.batch(args.eval_batch_size)

# loss_metric = tf.keras.metrics.Mean()
# epoch_bar = master_bar(range(1))
# pb_max_len = math.ceil(
# float(len(eval_features))/float(args.eval_batch_size))
# y_true = []
# y_pred = []
# label_map = {i : label for i, label in enumerate(label_list,1)}
# for epoch in epoch_bar:
#   for (input_ids, input_mask, segment_ids, valid_ids, label_ids) in progress_bar(batched_eval_data, total=pb_max_len, parent=epoch_bar):
#     logits = ner(input_ids, input_mask, segment_ids, valid_ids, training=False)
#     logits = tf.argmax(logits,axis=2)
#     for i, label in enumerate(label_ids):
#      temp_1 = []
#      temp_2 = []
#      for j,m in enumerate(label):
#        if j == 0:
#          continue
#        elif label_ids[i][j].numpy() == len(label_map):     
#          y_true.append(temp_1)
#          y_pred.append(temp_2)
#          break
#        else:
#          temp_1.append(label_map[label_ids[i][j].numpy()])
#          temp_2.append(label_map[logits[i][j].numpy()])

# report = classification_report(y_true, y_pred,digits=4)
# output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
# with open(output_eval_file, "w") as writer:
#   logger.info("***** Eval results *****")
#   logger.info("\n%s", report)
# writer.write(report)

In [27]:
def train_step(input_ids, input_mask, segment_ids, valid_ids, label_ids,label_mask):
  with tf.GradientTape() as tape:
    logits = ner(input_ids, input_mask,segment_ids, valid_ids, 
    training=True) #batchsize, max_seq_length, num_labels
    label_ids_masked = tf.boolean_mask(label_ids,label_mask)        
    logits_masked = tf.boolean_mask(logits,label_mask)
    loss = loss_fct(label_ids_masked, logits_masked)
  grads = tape.gradient(loss, ner.trainable_variables)
  optimizer.apply_gradients(list(zip(grads,       
  ner.trainable_variables)))
  return loss

In [32]:
# for epoch in epoch_bar:
#     for (input_ids, input_mask, segment_ids, valid_ids, 
#     label_ids,label_mask) in progress_bar(batched_train_data, 
#     total=pb_max_len, parent=epoch_bar):
#       loss = train_step(input_ids, input_mask, segment_ids,
#       valid_ids, label_ids,label_mask)
#       loss_metric(loss)
#       epoch_bar.child.comment = f'loss : {loss_metric.result()}'
#     loss_metric.reset_states()

In [31]:
# Train and Validation Features (text)
# X_train_val, X_test = pd.concat([split_train['text'], val_df['text']]).values, test_df['text'].values

In [77]:
# Train and Validation Labels
# y_train_val = label_encoder.fit_transform(pd.concat([train_df['label'], val_df['label']]))

In [78]:
# Split into new train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=0, stratify=y_train_val)